In [1]:
#USE HI-ML-UP-TO-DATE VENV 
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from pathlib import Path

import os
os.environ['HF_HOME'] = '/vol/biomedic3/bglocker/ugproj2324/nns20/CheXagent/.tuned_projection_model' ## THIS HAS TO BE BEFORE YOU IMPORT TRANSFORMERScd
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig, AdamW, get_cosine_schedule_with_warmup
from torchvision import transforms # THIS HAS TO BE AFTER THE ABOVE LINES


In [2]:

def setup_model() -> tuple:
    device = "cuda"
    dtype = torch.float16

    processor = AutoProcessor.from_pretrained("StanfordAIMI/CheXagent-8b", trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        "StanfordAIMI/CheXagent-8b", torch_dtype=dtype, trust_remote_code=True
    ).to(device)
    generation_config = GenerationConfig.from_pretrained("StanfordAIMI/CheXagent-8b")

    return processor, model, device, dtype, generation_config

def generate(images, prompt, processor, model, device, dtype, generation_config):
    inputs = processor(
        images=images, text=f" USER: <s>{prompt} ASSISTANT: <s>", return_tensors="pt"
    ).to(device=device, dtype=dtype)
    output = model.generate(**inputs, generation_config=generation_config)[0]
    response = processor.tokenizer.decode(output, skip_special_tokens=True)
    return response


In [3]:
processor, model, device, dtype, generation_config = setup_model()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Some weights of CheXagentForConditionalGeneration were not initialized from the model checkpoint at StanfordAIMI/CheXagent-8b and are newly initialized: ['gated_residual_layer.attention.in_proj_bias', 'gated_residual_layer.attention.in_proj_weight', 'gated_residual_layer.attention.out_proj.bias', 'gated_residual_layer.attention.out_proj.weight', 'gated_residual_layer.gate', 'gated_residual_layer.layer_norm.bias', 'gated_residual_layer.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
print(type(model))

<class 'transformers_modules.StanfordAIMI.CheXagent-8b.4934e91451945c8218c267aae9c34929a7677829.modeling_chexagent.CheXagentForConditionalGeneration'>


In [5]:
prompt = 'Describe the findings in the image.'
path = "/vol/biodata/data/chest_xray/mimic-cxr-jpg/files/p10/p10002428/s59891001/df59ff50-56f1fd83-f3664b8b-8e2c67dd-d5f0d039.jpg"

prompt = "Is the ILD on the left or the right of the patient?"
image_id = "e0dc2e79105ad93532484e956ef8a71a" # 688ecdb1a4e994d42b5a50a8c4a9736f
overall_test_png_image_dset_path = f'/vol/biodata/data/chest_xray/VinDr-CXR/1.0.0_png_512/raw/test/{image_id}.png'
paths = [path]#, test_png_dset_path]

transform = transforms.Compose([transforms.PILToTensor()])
images = [transform(Image.open(path).convert("RGB")) for path in paths]
response = generate(images, prompt, processor, model, device, dtype, generation_config)
print(response)

/vol/biomedic3/bglocker/ugproj2324/nns20/CheXagent/.tuned_projection_model/modules/transformers_modules/StanfordAIMI/CheXagent-8b/4934e91451945c8218c267aae9c34929a7677829/processing_chexagent.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(pixel_values) for pixel_values in encoding_image_processor["pixel_values"]]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Left


In [6]:
test_pathology_left_or_right = Path('/vol/biomedic3/bglocker/ugproj2324/nns20/datasets/VinDr-CXR/image_text_reasoning_datasets/test_pathology_left_or_right')
test_png_dset_path = Path('/vol/biodata/data/chest_xray/VinDr-CXR/1.0.0_png_512/raw/test')


In [7]:
import time
responses = []
total_correct = 0
with open(test_pathology_left_or_right) as f:
    f.readline() # skip header
    lines = f.readlines()
    for index, line in enumerate(lines):
        image_id,pathology,true_location = line.split(',')
        image = Image.open(test_png_dset_path / f"{image_id}.png").convert("RGB")
        prompt = f"Localize the {pathology}, is it on the RIGHT or the LEFT or the RIGHT AND LEFT of the given image?"

        start_time = time.time()
        response = generate([image], prompt,processor, model, device, dtype, generation_config)
        
        if time.time() - start_time > 10:
            print(f"Image {image_id} took {time.time() - start_time} seconds to generate")
            with open('inserted_layer_results', 'w') as f:
                f.write(f"super slow {image_id}")
        
        responses.append(f"{image_id},{pathology},{response}")

        if index % 100 == 0:
            with open('grl_RIGHT_or_LEFT_responses', 'w') as f:
                f.write('\n'.join(responses))
            print(f"Written up to index {index}")

/vol/biomedic3/bglocker/ugproj2324/nns20/CheXagent/.tuned_projection_model/modules/transformers_modules/StanfordAIMI/CheXagent-8b/4934e91451945c8218c267aae9c34929a7677829/processing_chexagent.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(pixel_values) for pixel_values in encoding_image_processor["pixel_values"]]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Written up to index 0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Written up to index 100


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Written up to index 2000


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Written up to index 2100


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
